In [8]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import plotly.express as px
import plotly.graph_objects as go
import torch

from utils.ucr_helpers import UCR_Data

In [5]:
dataset_names = [
    "ACSF1",
    "Adiac",
    "AllGestureWiimoteX",
    "AllGestureWiimoteY",
    "AllGestureWiimoteZ",
    "ArrowHead",
    "BME",
    "Beef",
    "BeetleFly",
    "BirdChicken",
    "CBF",
    "Car",
    "Chinatown",
    "ChlorineConcentration",
    "CinCECGTorso",
    "Coffee",
    "Computers",
    "CricketX",
    "CricketY",
    "CricketZ",
    "Crop",
    "DiatomSizeReduction",
    "DistalPhalanxOutlineAgeGroup",
    "DistalPhalanxOutlineCorrect",
    "DistalPhalanxTW",
    "DodgerLoopDay",
    "DodgerLoopGame",
    "DodgerLoopWeekend",
    "ECG200",
    "ECG5000",
    "ECGFiveDays",
    "EOGHorizontalSignal",
    "EOGVerticalSignal",
    "Earthquakes",
    "ElectricDevices",
    "EthanolLevel",
    "FaceAll",
    "FaceFour",
    "FacesUCR",
    "FiftyWords",
    "Fish",
    "FordA",
    "FordB",
    "FreezerRegularTrain",
    "FreezerSmallTrain",
    "Fungi",
    "GestureMidAirD1",
    "GestureMidAirD2",
    "GestureMidAirD3",
    "GesturePebbleZ1",
    "GesturePebbleZ2",
    "GunPoint",
    "GunPointAgeSpan",
    "GunPointMaleVersusFemale",
    "GunPointOldVersusYoung",
    "Ham",
    "HandOutlines",
    "Haptics",
    "Herring",
    "HouseTwenty",
    "InlineSkate",
    "InsectEPGRegularTrain",
    "InsectEPGSmallTrain",
    "InsectWingbeatSound",
    "ItalyPowerDemand",
    "LargeKitchenAppliances",
    "Lightning2",
    "Lightning7",
    "Mallat",
    "Meat",
    "MedicalImages",
    "MelbournePedestrian",
    "MiddlePhalanxOutlineAgeGroup",
    "MiddlePhalanxOutlineCorrect",
    "MiddlePhalanxTW",
    "MixedShapesRegularTrain",
    "MixedShapesSmallTrain",
    "MoteStrain",
    "NonInvasiveFetalECGThorax1",
    "NonInvasiveFetalECGThorax2",
    "OSULeaf",
    "OliveOil",
    "PLAID",
    "PhalangesOutlinesCorrect",
    "Phoneme",
    "PickupGestureWiimoteZ",
    "PigAirwayPressure",
    "PigArtPressure",
    "PigCVP",
    "Plane",
    "PowerCons",
    "ProximalPhalanxOutlineAgeGroup",
    "ProximalPhalanxOutlineCorrect",
    "ProximalPhalanxTW",
    "RefrigerationDevices",
    "Rock",
    "ScreenType",
    "SemgHandGenderCh2",
    "SemgHandMovementCh2",
    "SemgHandSubjectCh2",
    "ShakeGestureWiimoteZ",
    "ShapeletSim",
    "ShapesAll",
    "SmallKitchenAppliances",
    "SmoothSubspace",
    "SonyAIBORobotSurface1",
    "SonyAIBORobotSurface2",
    "StarLightCurves",
    "Strawberry",
    "SwedishLeaf",
    "Symbols",
    "SyntheticControl",
    "ToeSegmentation1",
    "ToeSegmentation2",
    "Trace",
    "TwoLeadECG",
    "TwoPatterns",
    "UMD",
    "UWaveGestureLibraryAll",
    "UWaveGestureLibraryX",
    "UWaveGestureLibraryY",
    "UWaveGestureLibraryZ",
    "Wafer",
    "Wine",
    "WordSynonyms",
    "Worms",
    "WormsTwoClass",
    "Yoga",
]
not_fixed_length = [
    "AllGestureWiimoteX",
    "AllGestureWiimoteY",
    "AllGestureWiimoteZ",
    "GestureMidAirD1",
    "GestureMidAirD2",
    "GestureMidAirD3",
    "GesturePebbleZ1",
    "GesturePebbleZ2",
    "MelbournePedestrian",
    "PLAID",
    "PickupGestureWiimoteZ",
    "ShakeGestureWiimoteZ",
]
dataset_names = [xi for xi in dataset_names if xi not in not_fixed_length]


In [3]:
# data = UCR_Data(name="Mallat")
# data = UCR_Data("StarLightCurves")
data = UCR_Data("Beef")

In [4]:
print(data.summary)
fig = data.plot_fig()
fig.show()

Number of classes: 5
Number of training samples: 30
Number of test samples: 30
Length of time series: 470


- Sample only from regions of high variance?

#### Create target context sets
Skip to next markdown header to load from memory

In [5]:
pd.Series(data.y_train).value_counts()

1    6
2    6
3    6
4    6
5    6
Name: count, dtype: int64

In [6]:
num_categories = data.n_classes
num_TS = data.y.shape[0]
TS_length = data.X.shape[1]
print(f"Number of Categories: {num_categories}")
print(f"Number of time series: {num_TS}")
print(f"Total length of each TS: {TS_length}")
X=data.X
y=data.y

Number of Categories: 5
Number of time series: 60
Total length of each TS: 470


In [7]:
context_size = min(max(int(num_TS*0.05), 5),50)
period = 50
stride = 10
num_neg_samples = max(context_size, 25)
batch_size = 64

print(f"Context Size: {context_size}, Period: {period}, Stride: {stride}")
print(f"Number of Negative Samples: {num_neg_samples}")

Context Size: 5, Period: 50, Stride: 10
Number of Negative Samples: 25


In [8]:
from utils.context import get_tgt_context_euclidean_multiprocess
positive_tgt_context_sets = get_tgt_context_euclidean_multiprocess(ts_array=X, m=period, k=context_size, stride=stride, z_normalize=True)
positive_tgt_context_sets += get_tgt_context_euclidean_multiprocess(ts_array=X, m=period, k=context_size, stride=stride, z_normalize=False)
print(len(positive_tgt_context_sets))

nearly returning


100%|██████████| 8/8 [00:00<00:00, 1368.28it/s]


nearly returning
5160


100%|██████████| 8/8 [00:00<00:00, 2069.35it/s]


In [9]:
negative_tgt_context_sets = get_tgt_context_euclidean_multiprocess(ts_array=X, m=period, k=num_neg_samples, stride=stride, z_normalize=True, top_k=False)
negative_tgt_context_sets += get_tgt_context_euclidean_multiprocess(ts_array=X, m=period, k=num_neg_samples, stride=stride, z_normalize=False, top_k=False)
print(len(negative_tgt_context_sets))

nearly returning


100%|██████████| 8/8 [00:00<00:00, 1259.79it/s]


nearly returning
5160


100%|██████████| 8/8 [00:00<00:00, 2031.39it/s]


In [15]:
490 in positive_tgt_context_sets[50034][1]

False

In [16]:
490 in negative_tgt_context_sets[50035][1]

False

In [14]:
# tgt_context_sets += tgt_context_sets_unnormalized

#### Save and load target context sets

In [24]:
#-- SAVE
from utils.helpers_general import save_idx_combinations_json

save_fname = f"StarLightCurves_E_znorm_p{period}_s{stride}_c{context_size}.json"
print(save_fname)
# save_idx_combinations_json(idx_combinations=tgt_context_sets, fname=f"target_context_sets/{save_fname}")

StarLightCurves_E_znorm_p5_s5_c10.json


In [43]:
#-- LOAD
from utils.helpers_general import read_idx_combinations_json
tgt_context_sets = read_idx_combinations_json(fname="target_context_sets/StarLightCurves_E_p10_s10_c50.json")

## Get the training dataset

In [10]:
from collections import defaultdict
def get_class_dict(data, train=True):
    class_dict = defaultdict(list)
    if train:
        for i in range(data.X_train.shape[0]):
            class_dict[data.y_train[i]].append(i)
    else:
        for i in range(data.X.shape[0]):
            class_dict[data.y[i]].append(i)
    return {k: set(v) for k, v in class_dict.items()}
# class_dict = get_class_dict(data, train=True)
class_dict = get_class_dict(data, train=False)

In [57]:
if False:
    positive_class_samples_by_index = {i:list(class_dict[data.y_train[i]]-{i}) for i in range(len(data.y_train))}
else:
    positive_class_samples_by_index = {i:list(class_dict[data.y[i]]-{i}) for i in range(len(data.y))}
def get_positive_class_lookup(positive_class_samples_by_index):
    minimum_number_in_class = min([len(v) for k,v in positive_class_samples_by_index.items()])
    positive_class_lookup = np.array([np.random.choice(v, size=minimum_number_in_class) for k,v in positive_class_samples_by_index.items()])
    return torch.tensor(positive_class_lookup).long()
positive_class_lookup = get_positive_class_lookup(positive_class_samples_by_index)

In [58]:
negative_class_samples_by_index = {}
for i in range(num_TS):
    others = set(class_dict.keys()) - set(data.y[i])
    others_indices = set.union(*[class_dict[i] for i in others])
    negative_class_samples_by_index[i] = list(others_indices)

In [20]:
negative_class_lookup = get_positive_class_lookup(negative_class_samples_by_index)

In [10]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
    def __init__(self, positive_similarity, negative_similarity, anchor):
        self.positive_similarity = positive_similarity
        self.negative_similarity = negative_similarity
        self.anchor = anchor

    def __getitem__(self, i):
        return (self.positive_similarity[i], self.negative_similarity[i], self.anchor[i])

    def __len__(self):
        return len(self.anchor)

def get_train_loader(idx_combinations, negative_idx_combinations, batch_size=64):
    # Create list of x values and y values from list of tuples in idx_combinations
    positive_similarity_vals = np.array([idx_combinations[i][1] for i in range(len(idx_combinations))])
    negative_similarity_vals = np.array([negative_idx_combinations[i][1] for i in range(len(negative_idx_combinations))])
    anchor_vals = np.array([idx_combinations[i][0] for i in range(len(idx_combinations))])

    # Convert x and y values to tensors
    positive_tensor = torch.from_numpy(positive_similarity_vals).long()
    negative_tensor = torch.from_numpy(negative_similarity_vals).long()
    anchor_tensor = torch.from_numpy(anchor_vals).long()

    # Create a dataset from the tensors
    train_data = CustomDataset(positive_tensor, negative_tensor, anchor_tensor)

    # Create a data loader for the dataset with specified batch size and shuffle options
    train_loader = DataLoader(
        dataset=train_data, batch_size=batch_size, shuffle=True, drop_last=True
    )

    return train_loader

In [11]:
import torch
import torch.nn as nn
# from utils.training_helpers import get_train_loader
train_loader = get_train_loader(idx_combinations=positive_tgt_context_sets, negative_idx_combinations=negative_tgt_context_sets)

In [14]:
len(train_loader)

7350

In [28]:
import numpy as np
import torch
import torch.nn as nn


class BatchContrastiveUnsupervised(nn.Module):
    def __init__(self, n_time_series: int, embedding_dim: int):
        """_summary_

        Args:
            n_time_series (int): Number of time series. I.e. first dimension of embedding matrix
            embedding_dim (int): Dimension of embeddings to be learned
        """
        super(
            BatchContrastiveUnsupervised, self
        ).__init__()  # -- This line calls the parent class nn.Module
        # -- Only use one embedding matrix here
        self.embeddings = nn.Embedding(n_time_series, embedding_dim)
        self.log_sigmoid = nn.LogSigmoid()
        self.criterion = nn.BCEWithLogitsLoss()

    def forward(
        self, positive_similarity_batch, negative_similarity_batch, anchor_batch
    ):
        positive_embeddings = self.embeddings(
            positive_similarity_batch
        )  # -- Shape (batch_size, context_size, embedding_dim)
        negative_embeddings = self.embeddings(
            negative_similarity_batch
        )  # -- Shape (batch_size, context_size, embedding_dim)
        anchor_embeddings = self.embeddings(
            anchor_batch
        )  # -- Shape (batch_size, embedding_dim)
        anchor_embeddings = torch.unsqueeze(
            anchor_embeddings, dim=1
        )  # -- Shape -> (batch_size, 1, embedding_dim)

        self.p, self.n, self.a = (
            positive_embeddings,
            negative_embeddings,
            anchor_embeddings,
        )

        positive_score = torch.einsum(
            "bce,bie->bc", [positive_embeddings, anchor_embeddings]
        )
        negative_score = torch.einsum(
            "bce,bie->bc", [negative_embeddings, anchor_embeddings]
        )
        positive_samples_logits = -torch.sum(positive_score, dim=1)
        negative_samples_logits = torch.sum(negative_score, dim=1)
        # positive_samples_logits = -torch.sum(self.log_sigmoid(positive_score), dim=1)
        # negative_samples_logits = -torch.sum(self.log_sigmoid(-negative_score), dim=1)
        return positive_samples_logits, negative_samples_logits

In [29]:
# from model_classes.contrastive import BatchContrastiveUnsupervised

model = BatchContrastiveUnsupervised(n_time_series=num_TS, embedding_dim=20)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
lmda = 0

In [ ]:

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
lmda = 0.5
# Choose device based on availability of CUDA
# device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
device = torch.device("cpu")
print("Running on: ", device)

# Move model to the chosen device
model = model.to(device)

# Setting the model to training mode
model.train()

num_epochs = 16
for epoch in tqdm(range(num_epochs)):  # num_epochs is the number of epochs you want to train the model


    running_loss = 0.0
    for i, (positive_similarity_batch, negative_similarity_batch, anchor_batch) in tqdm(enumerate(train_loader), total=len(train_loader), disable=True):
        # Move data to the chosen device
        positive_similarity_batch = positive_similarity_batch.to(device)
        negative_similarity_batch = negative_similarity_batch.to(device)
        # -- Test if we use actual data from same class as positives
        # if i%500==0:
        #     positive_class_lookup = get_positive_class_lookup(positive_class_samples_by_index)
        #     negative_class_lookup = get_positive_class_lookup(negative_class_samples_by_index)
        # positive_similarity_batch = positive_class_lookup[anchor_batch].to(device)
        # negative_similarity_batch = negative_class_lookup[anchor_batch].to(device)


        anchor_batch = anchor_batch.to(device)

        # Forward pass
        positive_similarity_logits, negative_similiarity_logits = model(positive_similarity_batch, negative_similarity_batch, anchor_batch)
        # Compute Loss
        positive_similarity_loss = criterion(positive_similarity_logits, torch.ones_like(positive_similarity_logits))
        negative_similarity_loss = criterion(negative_similiarity_logits, torch.zeros_like(negative_similiarity_logits))
        # Regularization Loss - Make all embeddings have norm of one
        if lmda>0:regularization_loss = lmda * torch.sum(torch.abs(model.embeddings.weight.norm(dim=1) - torch.ones(num_TS)))
        else: regularization_loss=0
        # Backward Pass
        optimizer.zero_grad()
        negative_similarity_weight = context_size/(context_size+num_neg_samples)
        positive_similarity_weight = num_neg_samples/(context_size+num_neg_samples)
        loss = positive_similarity_weight*positive_similarity_loss + negative_similarity_weight*negative_similarity_loss
        loss += regularization_loss
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # print statisktics
        running_loss += loss.item()
    if epoch % 1 == 0:
        print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss))
        running_loss = 0.0

print('Finished Training')


Running on:  cpu


  6%|▋         | 1/16 [00:11<02:58, 11.88s/it]

[1,  7350] loss: 2477511.417


 12%|█▎        | 2/16 [00:23<02:40, 11.48s/it]

[2,  7350] loss: 7648.578


...

[7,  7350] loss: 7122.196


 50%|█████     | 8/16 [01:29<01:30, 11.28s/it]

[8,  7350] loss: 7073.613


: 

: 

In [1]:
regularization_loss

NameError: name 'regularization_loss' is not defined

In [26]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

smote = False
embeddings = model.embeddings.weight.detach().numpy()
train_size = data.X_train.shape[0]
embeddings_train = embeddings[:train_size, :]
embeddings_test = embeddings[train_size:, :]
y_train = data.y[:train_size]
y_test = data.y[train_size:]


# classifier = SVC(kernel='rbf')
classifier = MLPClassifier(hidden_layer_sizes=[100], max_iter=1000)
# classifier = AdaBoostClassifier(base_estimator=SVC(kernel='rbf', probability=True))
if smote:
    sm = SMOTE(random_state=42)
    X_train_oversampled, y_train_oversampled = sm.fit_resample(embeddings_train, y_train)
    classifier.fit(X_train_oversampled, y_train_oversampled)
else:
    classifier.fit(embeddings_train, y_train)
y_preds = classifier.predict(embeddings_test)
print(classification_report(y_true=y_test, y_pred=y_preds))

              precision    recall  f1-score   support

           1       0.17      0.09      0.11       294
           2       0.14      0.13      0.13       292
           3       0.13      0.13      0.13       294
           4       0.12      0.25      0.16       289
           5       0.04      0.00      0.01       298
           6       0.07      0.05      0.06       294
           7       0.13      0.23      0.16       291
           8       0.14      0.12      0.13       293

    accuracy                           0.12      2345
   macro avg       0.12      0.12      0.11      2345
weighted avg       0.12      0.12      0.11      2345



### Training

In [14]:
num_neg_samples
context_size

50

In [15]:

from tqdm import tqdm
import cProfile

# Start the profiler
profiler = cProfile.Profile()
profiler.enable()
losses = []
for epoch in tqdm(range(10)):
    neg_sample_idxs = get_neg_sample_idxs(individual_probs_all=individual_probs_all, num_neg_samples=num_neg_samples, full_distribution=False, random=False)
    # neg_sample_idxs = get_neg_sample_idxs(individual_probs_all=individual_probs_all, num_neg_samples=num_neg_samples, full_distribution=True)
    # neg_sample_idxs = get_neg_sample_idxs(individual_probs_all=individual_probs_all, num_neg_samples=num_neg_samples, random=True)
    epoch_loss = 0
    for context_batch, query_batch in train_loader:
        context_size = context_batch.shape[1]
        # context_negative_batch = get_context_negative_batch(query_batch=query_batch, context_batch=context_batch, neg_sample_idxs=neg_sample_idxs)
        negative_batch = get_negative_batch(query_batch=query_batch, neg_sample_idxs=neg_sample_idxs)
        context_score, negative_score = model(context_batch, negative_batch, query_batch)
        # context_score, negative_score = criterion(raw_similarities, context_size=context_size)
        context_loss = criterion(context_score, torch.ones_like(context_score))
        negative_loss = criterion(negative_score, torch.zeros_like(negative_score))

        regularization_loss = lmda * model.embeddings.weight.norm(dim=1).mean()
        # -- Backward Pass
        optimizer.zero_grad()
        context_weight = context_size/(context_size+num_neg_samples)
        negative_weight = num_neg_samples/(context_size+num_neg_samples)
        loss = context_weight*context_loss + negative_weight*negative_loss
        loss.backward()
        # -- Update Gradient
        optimizer.step()
        epoch_loss += loss.item()
    losses.append(epoch_loss)

    # Print loss and accuracy on validation set
    if epoch%2==0:
        with torch.no_grad():
            print("="*25)
            print(epoch)
            print(epoch_loss)
            # embeddings = model.embeddings.weight.detach().numpy()

            # get_sector_score(embedding_matrix=embeddings[train_size:,:], sectors=data.y[train_size:], smote=True)

            # # Early stopping check
            # if val_acc > best_val_acc:
            #     best_val_acc = val_acc
            #     early_stopping_counter = 0
            # else:
            #     early_stopping_counter += 1

            # if early_stopping_counter >= early_stopping_patience:
            #     print("Early stopping at epoch: {} with best validation accuracy: {:.2f}%".format(epoch, best_val_acc*100))
            #     break

profiler.disable()

 10%|█         | 1/10 [00:12<01:51, 12.39s/it]

0
6096.0801737606525


 30%|███       | 3/10 [00:36<01:26, 12.30s/it]

2
2849.627938747406


 50%|█████     | 5/10 [01:01<01:01, 12.31s/it]

...

 70%|███████   | 7/10 [01:26<00:37, 12.33s/it]

6
2658.953069806099


 90%|█████████ | 9/10 [01:50<00:12, 12.35s/it]

8
2622.775785058737


100%|██████████| 10/10 [02:03<00:00, 12.33s/it]


In [14]:
profiler.print_stats(sort='cumulative')

         57246692 function calls (51152606 primitive calls) in 211.068 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000  211.067   70.356 interactiveshell.py:3424(run_code)
        3    0.000    0.000  211.067   70.356 {built-in method builtins.exec}
        1    2.050    2.050  211.067  211.067 3924937432.py:8(<module>)
    88314    0.153    0.000   85.097    0.001 _tensor.py:429(backward)
    88314    0.218    0.000   84.933    0.001 __init__.py:103(backward)
    88314   84.262    0.001   84.262    0.001 {method 'run_backward' of 'torch._C._EngineBase' objects}
529884/264942    0.503    0.000   36.869    0.000 module.py:1188(_call_impl)
    88314    0.385    0.000   28.979    0.000 optimizer.py:135(wrapper)
    88314    0.245    0.000   27.391    0.000 optimizer.py:19(_use_grad)
    88314    0.375    0.000   26.934    0.000 adam.py:168(step)
    88314    0.407    0.000   26.460    0.000 adam.p

In [16]:
import plotly.express as px
px.scatter(losses)

### Evaluation

In [17]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

smote = False
embeddings = model.embeddings.weight.detach().numpy()
train_size = data.X_train.shape[0]
embeddings_train = embeddings[:train_size, :]
embeddings_test = embeddings[train_size:, :]
y_train = data.y[:train_size]
y_test = data.y[train_size:]


# classifier = SVC(kernel='rbf')
classifier = MLPClassifier(hidden_layer_sizes=[100])
# classifier = AdaBoostClassifier(base_estimator=SVC(kernel='rbf', probability=True))
if smote:
    sm = SMOTE(random_state=42)
    X_train_oversampled, y_train_oversampled = sm.fit_resample(embeddings_train, y_train)
    classifier.fit(X_train_oversampled, y_train_oversampled)
else:
    classifier.fit(embeddings_train, y_train)
y_preds = classifier.predict(embeddings_test)
print(classification_report(y_true=y_test, y_pred=y_preds))

/opt/anaconda3/envs/embeddings/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



              precision    recall  f1-score   support

           1       1.00      0.95      0.98       294
           2       0.95      0.99      0.97       292
           3       0.59      0.92      0.72       294
           4       0.95      1.00      0.98       289
           5       0.92      0.16      0.27       298
           6       0.93      0.97      0.95       294
           7       0.81      0.98      0.89       291
           8       0.99      0.95      0.97       293

    accuracy                           0.86      2345
   macro avg       0.89      0.87      0.84      2345
weighted avg       0.89      0.86      0.84      2345



In [18]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true=y_test, y_pred=y_preds))

[[280   0   0  14   0   0   0   0]
 [  0 290   0   0   0   0   1   1]
 [  0   0 271   0   3  20   0   0]
 [  0   0   0 289   0   0   0   0]
 [  0   0 186   0  47   3  62   0]
 [  0   0   6   0   0 286   2   0]
 [  0   3   0   0   1   0 285   2]
 [  0  12   0   0   0   0   3 278]]


In [57]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.decomposition import PCA

# generate some sample data
embeddings = model.embeddings.weight.detach().numpy()

# apply PCA to reduce the dimensionality of the data to 2D
pca = PCA(n_components=3)
pca.fit(embeddings)
pca_data = pca.transform(embeddings)

# generate some sample class labels
labels = data.y

# convert the PCA data, class labels, and entity names to a pandas DataFrame for plotting with plotly
df = pd.DataFrame({'x': pca_data[:, 0], 'y': pca_data[:, 1], 'label': labels})

# plot the data using plotly, colored by the class labels and with entity names in the hover label
fig = px.scatter(df, x='x', y='y', color='label', hover_name='label')
fig.update_layout(template='plotly_dark')
fig.show()

## Regualar Baselines

In [30]:
X_train_df = UCR_Data.sktime_from_numpy(data.X_train)
X_test_df = UCR_Data.sktime_from_numpy(data.X_test)


In [31]:
from sktime.transformations.panel.catch22 import Catch22

catch22 = Catch22()

train_catch22_features = catch22.fit_transform(X_train_df)
test_catch22_features = catch22.transform(X_test_df)


In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
# temp_model = DecisionTreeClassifier()
temp_model = RandomForestClassifier()
temp_model.fit(train_catch22_features, data.y_train)

print(classification_report(y_true=data.y_test, y_pred=temp_model.predict(test_catch22_features)))

              precision    recall  f1-score   support

           1       0.97      0.71      0.82       294
           2       1.00      0.99      1.00       292
           3       0.86      0.95      0.90       294
           4       0.77      0.98      0.86       289
           5       0.99      0.79      0.88       298
           6       0.94      0.95      0.95       294
           7       0.96      1.00      0.98       291
           8       0.94      0.99      0.97       293

    accuracy                           0.92      2345
   macro avg       0.93      0.92      0.92      2345
weighted avg       0.93      0.92      0.92      2345



### Predict based on predict_proba output from both models

In [ ]:
train_probs_combined = np.concatenate([temp_model.predict_proba(train_catch22_features),classifier.predict_proba(embeddings_train)], axis=1)
test_probs_combined = np.concatenate([temp_model.predict_proba(test_catch22_features),classifier.predict_proba(embeddings_test)], axis=1)

clf = RandomForestClassifier()
clf.fit(train_probs_combined, data.y_train)
print(classification_report(y_true=data.y_test, y_pred=clf.predict(test_probs_combined)))

### Directly concat catch22 and embeddings

In [34]:

combined_train = np.concatenate([train_catch22_features, embeddings[:data.X_train.shape[0]]], axis=1)
combined_test = np.concatenate([test_catch22_features, embeddings[:data.X_test.shape[0]]], axis=1)


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
temp_model = MLPClassifier()
from sklearn.preprocessing import StandardScaler

scaled = True
if scaled is True:
    scaler = StandardScaler()
    combined_train_scaled = scaler.fit_transform(combined_train)
    temp_model.fit(combined_train_scaled, data.y_train)
    print(classification_report(y_true=data.y_test, y_pred=temp_model.predict(scaler.transform(combined_test))))
else:
    temp_model.fit(combined_train, data.y_train)
    print(classification_report(y_true=data.y_test, y_pred=temp_model.predict(combined_test)))
